In [62]:
import os
import csv
import time
import json
import requests
import pathlib
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

In [63]:
load_dotenv()

client = OpenAI()
STOCKS_API_KEY = os.getenv('STOCKS_API_KEY')

In [64]:
def create_assistant(vector_store_id) -> str:
    tools = [
        {"type": "code_interpreter"},
        {"type": "file_search"},
        {
            "type": "function",
            "function": {
                "name": "get_stock_price",
                "description": "Get the latest stock price for a symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The stock symbol (e.g., AAPL, MSFT, TSLA)"
                        }
                    },
                    "required": ["symbol"]
                }
            }
        }
    ]

    assistant = client.beta.assistants.create(
        name="Stock Market Analyser",
        instructions="""You are a financial assistant that helps users analyse stock data.
You can interpret stock data like date, open, high, low, close, volume.
You can also calculate technical indicators (SMA, RSI, MACD, etc.), and plot relevant charts.
You can fetch real-time stock data if the user requests it using the get_stock_price function.
After fetching the data, even if the user doesn't ask, give your opinion on the data received; based on the intraday data you receive, provide one suggestion for the user's investment decision (e.g., hold, watch, buy, sell), and explain why.
Use uploaded strategy documents to provide smart investment advice based on common patterns, rules, principles and anything that you find in those files, also use your own strategies.""",
        tools=tools,
        tool_resources={"file_search": {"vector_store_ids": [vector_store_id]},},
        model="gpt-4o-mini",
    )

    return assistant.id

In [65]:
def get_assistant(vector_store_id) -> str:
    assistants = client.beta.assistants.list()
    if len(assistants.data) == 0:
        return create_assistant(vector_store_id)
    return assistants.first_id

In [66]:
def create_thread() -> str:
    return client.beta.threads.create().id

In [67]:
def generate_csv(timeseries_data, file_name: str) -> None:
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['datetime', 'open', 'high', 'low', 'close', 'volume'])
        for date_time, values in timeseries_data.items():
            writer.writerow([
                date_time,
                values['o'],
                values['h'],
                values['l'],
                values['c'],
                values['v']
            ])

In [68]:
def generate_plot(timeseries_data, file_name: str) -> None:
    date_times = [datetime.strptime(ts, '%H:%M') for ts in timeseries_data.keys()]

    # Extract values and convert to numeric
    open_ = [float(v["o"]) for v in timeseries_data.values()]
    high = [float(v["h"]) for v in timeseries_data.values()]
    low = [float(v["l"]) for v in timeseries_data.values()]
    close = [float(v["c"]) for v in timeseries_data.values()]
    volume = [int(v["v"]) for v in timeseries_data.values()]

    # Create the figure and first axis
    fig, ax1 = plt.subplots(figsize=(22, 11))

    # Plot lines for price data
    ax1.plot(date_times, close, marker='o', ms=2, label='Close')
    ax1.plot(date_times, high, marker='o', ms=2, label='High')
    ax1.plot(date_times, low, marker='o', ms=2, label='Low')
    ax1.plot(date_times, open_, marker='o', ms=2, label='Open')
    ax1.set_xlabel("Time")
    ax1.set_ylabel("Price")
    ax1.tick_params(axis='x', rotation=25)
    ax1.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[0, 15, 30, 45]))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1.grid(True, axis='x', linestyle='--', alpha=0.5)

    # Create a secondary y-axis for volume bars
    ax2 = ax1.twinx()
    ax2.bar(date_times, volume, width=0.002, alpha=0.3, color='gray', label='Volume')
    ax2.set_ylabel("Volume")
    y_ticks = [int(x) for x in np.linspace(0, max(volume), 7)]
    ax2.set_yticks(y_ticks)
    ax2.ticklabel_format(style='plain', axis='y')
    ax2.grid(True, axis='y', linestyle='--', alpha=0.5)

    # Merge legends from both axes
    lines_labels = ax1.get_legend_handles_labels()
    bars_labels = ax2.get_legend_handles_labels()
    ax1.legend(lines_labels[0] + bars_labels[0], lines_labels[1] + bars_labels[1], loc='upper left')

    # Save
    plt.title("Price and Volume Over Time")
    plt.tight_layout()
    plt.grid(True)
    plt.savefig(file_name)
    plt.close(fig)

In [69]:
def get_stock_price(symbol: str) -> dict:
    """
    Fetches the most recent stock price for a given symbol using Alpha Vantage.

    :param symbol: The stock ticker (e.g.: 'AAPL', 'GOOGL').

    :return: A dictionary containing the latest stock prices and timestamps.
    """
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": "5min",
        "apikey": STOCKS_API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    reformatted_result = {"Meta Data": {k[3:]: v for k, v in data["Meta Data"].items() if not "Output" in k}}
    main_key = list(data.keys())
    main_key.remove("Meta Data")
    main_key = main_key[0]
    timeseries_dict = {_k.split()[1][:-3]: {k[3:4]: v for k, v in _v.items()} for _k, _v in data[main_key].items()}
    reformatted_result["Time Series"] = timeseries_dict
    sorted_data = dict(reversed(timeseries_dict.items()))

    generate_csv(sorted_data,
                 f"timeseries_{reformatted_result["Meta Data"]["Symbol"]}_{reformatted_result["Meta Data"]["Last Refreshed"].split()[0]}.csv")
    generate_plot(sorted_data,
                  f"timeseries_{reformatted_result["Meta Data"]["Symbol"]}_{reformatted_result["Meta Data"]["Last Refreshed"].split()[0]}.png")

    return {
        "Info": f"Intraday ({reformatted_result["Meta Data"]["Last Refreshed"].split()[0]}) (5min) open, high, low, close prices and volume",
        "Symbol": reformatted_result["Meta Data"]["Symbol"],
        "Time Series": sorted_data
    }

In [70]:
def get_assistant_response(assistant_id: str, thread_id: str, message: str):
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message
    )

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run_status.status == "completed":
            break
        elif run_status.status == "requires_action":
            required_actions = run_status.required_action.submit_tool_outputs.model_dump()
            tool_outputs = []

            for action in required_actions["tool_calls"]:
                func_name = action['function']['name']
                arguments = json.loads(action['function']['arguments'])

                if func_name == "get_stock_price":
                    output = get_stock_price(
                        symbol=arguments["symbol"],
                    )
                    tool_outputs.append({
                        "tool_call_id": action['id'],
                        "output": json.dumps(output)
                    })
                else:
                    raise ValueError(f"Unknown function: {func_name}")

            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
        elif run_status.status == "failed":
            return "Run failed."
        time.sleep(1)

    messages = client.beta.threads.messages.list(thread_id=thread_id)

    for msg in messages.data:
        if msg.role == "assistant":
            for content in msg.content:
                if content.type == "text":
                    return content.text.value
    return "FAILED"

In [71]:
def cleanup_all(d_assistants=0, d_files=0, d_vs=0):
    def delete_assistants():
        try:
            assistants = client.beta.assistants.list()
            if len(assistants.data) == 0:
                print("No assistants found")
            for assistant in assistants.data:
                try:
                    client.beta.assistants.delete(assistant.id)
                    print(f"Deleted assistant: {assistant.id}")
                except Exception as e:
                    print(f"Could not delete assistant {assistant.id}: {e}")
        except Exception as e:
            print(f"Failed to list/delete assistants: {e}")

    def delete_files():
        try:
            files = client.files.list()
            if len(files.data) == 0:
                print("No files found")
            for file in files.data:
                try:
                    client.files.delete(file.id)
                    print(f"Deleted file: {file.id}")
                except Exception as e:
                    print(f"Could not delete file {file.id}: {e}")
        except Exception as e:
            print(f"Failed to list/delete files: {e}")

    def delete_vs():
        try:
            v_stores = client.beta.vector_stores.list()
            if len(v_stores.data) == 0:
                print("No vector stores found")
            for vs in v_stores:
                try:
                    client.beta.vector_stores.delete(vs.id)
                    print(f"Deleted vector store {vs.id}")
                except Exception as e:
                    print(f"Could not delete store {vs.id}: {e}")
        except Exception as _:
            pass

    not d_assistants or delete_assistants()
    not d_files or delete_files()
    not d_vs or delete_vs()

In [74]:
def build_vector_store(dir_path: str, store_name: str) -> str:
    file_paths = list(pathlib.Path(dir_path).glob("*.md"))
    file_streams = [open(p, "rb") for p in file_paths]

    vs = client.beta.vector_stores.create(name=store_name)
    batch = client.beta.vector_stores.file_batches.upload_and_poll(vector_store_id=vs.id, files=file_streams)
    assert batch.status == "completed", "Vector-store upload failed"
    return vs.id

In [75]:
vs_id = build_vector_store("RAG", "RAG_strategies")
assistant_id = get_assistant(vs_id)
thread_id = create_thread()

In [76]:
def main():
    print("Start chatting with the assistant. Type 'exit' to quit.")
    while True:
        user_input = input("User:\n")
        if "exit".startswith(user_input.lower()):
            print("Exiting the interaction. Goodbye!")
            break
        print("User:", user_input, sep="\n")
        try:
            response = get_assistant_response(assistant_id, thread_id, user_input)
            print("\nAssistant:")
            print(response)
            print("\n" + "-" * 70 + "\n")
        except Exception as e:
            print(f"An error occurred: {e}")


main()

Start chatting with the assistant. Type 'exit' to quit.
User:
What’s the current stock price for Apple?

Assistant:
The current stock price for Apple (AAPL) is approximately **$207.9100**. 

### Key Intraday Data:
- **Open Price:** $207.7500
- **High Price:** $208.1500
- **Low Price:** $207.6401
- **Latest Close Price:** $207.9100
- **Volume:** 7,636 shares traded at the last recorded price

### Investment Suggestion:
Given the latest price action where the stock opened at $207.7500 and closed at $207.9100, there seems to be an upward trend for the day. The high of $208.1500 indicates potential strength, and if the price holds above $207.50, it might continue to trend upwards.

**Recommendation: Buy** – If you're considering a long-term investment, it's a favorable time to buy, especially if it dips close to the $207 mark again. However, remain cautious and set a stop-loss just below $207 to mitigate risk, as the market can be volatile. Always consider your financial objectives and ris

In [77]:
# TODO: Run the line below, after done with everything
cleanup_all(1, 1, 1)

Deleted assistant: asst_3ou7UGuadYfDfYqwUCNrfbWA
Deleted file: file-2HbMZXMosv71FnfKcwXX7a
Deleted file: file-XbmbosyJNABUQWHPSxyicR
Deleted file: file-GTHwf9EUR82H9mfemQKdWB
Deleted file: file-WKDFhrXSo355aNywWVxcbC
Deleted file: file-E3H5k2yNx5P58DxYmoSDuB
Deleted file: file-DprjSu6GUDui2rkzUqLsf2
Deleted file: file-GYgBDCcEsarzvCX2TWeXww
Deleted file: file-52faixmKWQdvk8mF5GXvWU
Deleted vector store vs_680b473bf92881918f27843f3a8a33b2
